In [1]:
import tifffile
from matplotlib import pyplot as plt
import numpy as np
import sys
import os

utils_path = os.path.abspath(os.path.join('..', 'utils'))

if utils_path not in sys.path:
    sys.path.append(utils_path)
import file_utilities as fu
from QuestionMaster import Questions

In [4]:
import tifffile
from matplotlib import pyplot as plt
import numpy as np
import sys
import os

utils_path = os.path.abspath(os.path.join('..', 'utils'))

if utils_path not in sys.path:
    sys.path.append(utils_path)
import file_utilities as fu
from QuestionMaster import Questions

file_path = fu.choose_file()

# Open the TIFF image
bg_remove = tifffile.imread(file_path)

Selected File: C:/Users/Ding-Yang/Desktop/Thesis code/Axons-extraction/example data/less_intensive/top_hat/top_hat.tiff


# Choose your sigma for initial segmentation

In [2]:
#Simetime it doesn't shows the image in jupyter lab. Use jupyter notebook instead.
from skimage.io import imread
import pyclesperanto_prototype as cle  # version 0.24.2
import napari_simpleitk_image_processing as nsitk  # version 0.4.5
import napari
from skimage.filters import gaussian, threshold_otsu, sobel
from skimage.measure import label
import stackview

def my_custom_code(image, sigma:float = 1, show_labels: bool = True):
    sigma =sigma
    binary_image =  cle.greater_or_equal_constant(image, None, sigma)
    edge_image = sobel(binary_image)
    
    if show_labels:
        return label(binary_image)
    else:
        return edge_image * 255 + image 

stackview.interact(my_custom_code, bg_remove, sigma=(0.0, 40, 0.5),zoom_factor =0.5)

# Determind whether you want to cut your images into small patches

In [20]:
from patchify import patchify, unpatchify

image_dataset = np.array(bg_remove)
# Ask for sigma value first
user_input_sigma = float(input("Insert your final sigma (Use the previous as reference): "))

# Preprocess image dataset
binary_volume = cle.greater_or_equal_constant(image_dataset, None, user_input_sigma)  # Use napari assistant for binarization
markers, num_features = ndi.label(binary_volume)

# Filter labels, keeping only those larger than the minimum size
min_label_size = 1000
filtered_markers = np.zeros_like(markers)
unique_labels, label_counts = np.unique(markers, return_counts=True)
for label, count in zip(unique_labels, label_counts):
    if count >= min_label_size:
        filtered_markers[markers == label] = label


Insert your final sigma (Use the previous as reference):  3.25


In [21]:
# Ask if the user wants to cut the image into patches
question = Questions("Do you want to cut your image into small patches? (If your image has intensive axon, recommend yes.)")
user_input_cut = question.ask_question(input_type='yes_no')

if user_input_cut:
    valid_patch_size = False  # Initialize a flag for valid input

    while not valid_patch_size:
        # Ask for patch size
        question = Questions("What size do you want to cut the image into small patches? (The smaller you cut, more time you need to do the correction. Recommend:1/8 ~ 1/2 width)")
        patch_size = question.ask_question(input_type='int')

        # Check if the patch size divides the image size evenly
        if image_dataset.shape[1] % patch_size == 0:
            valid_patch_size = True  # Set the flag to True if input is valid

            # Create patches for image_dataset
            img_patches = patchify(image_dataset, (image_dataset.shape[0], patch_size, patch_size), step=patch_size)
            image_dataset = np.reshape(img_patches, (-1, img_patches.shape[3], img_patches.shape[4], img_patches.shape[5]))
            
            # Create patches for mask_dataset
            mask_patches = patchify(filtered_markers, (filtered_markers.shape[0], patch_size, patch_size), step=patch_size)
            mask_dataset = np.reshape(mask_patches, (-1, mask_patches.shape[3], mask_patches.shape[4], mask_patches.shape[5]))
            
            print(f"You get {image_dataset.shape[0]} patches with image size {image_dataset.shape[1:4]}. The complete shape is {image_dataset.shape}")  # n_patches, x, y, z
        else:
            print(f"The size {patch_size} could not divide {image_dataset.shape[1]} evenly. Please enter a valid size.")
else:
    # No patching, use single image
    mask_dataset = np.expand_dims(filtered_markers, axis=0)
    image_dataset = np.expand_dims(image_dataset, axis=0)
    print(f"You get 1 patch with image size {image_dataset.shape}") 

def laplacian_sharpening_3d(image):
        laplacian = ndi.laplace(image)
        sharpened = image - laplacian
        return np.clip(sharpened, 0, 255)
sharpened_img_dataset = np.zeros_like(image_dataset, dtype=np.uint8)
num_images, depth, height, width = image_dataset.shape  # 確保正確的形狀

# Apply laplacian sharpening to each 3D image and store the results
for i in range(num_images):
        sharpened_img_dataset[i] = laplacian_sharpening_3d(image_dataset[i])
        

Do you want to cut your image into small patches? (If your image has intensive axon, recommend yes.) (y/n):  y
What size do you want to cut the image into small patches? (The smaller you cut, more time you need to do the correction. Recommend:1/8 ~ 1/2 width):  1024


You get 1 patches with image size (17, 1024, 1024). The complete shape is (1, 17, 1024, 1024)


# Start manual correction

In [8]:
from napari_editor import ImageEditor
editor = ImageEditor(image_dataset, sharpened_img_dataset, mask_dataset)
editor.start()

Enter the image index to start with (0 to 0):  0


## If you want to take a rest and continue to correct the data next time, you can use this to load the files

In [7]:
from manual_correction_data_loader import choose_directory_and_load_datasets

# Usage example:
datasets, missing_files = choose_directory_and_load_datasets()
if datasets:
    if 'image_dataset.tiff' in datasets:
        image_dataset = datasets['image_dataset.tiff']
    if 'sharpened_img_dataset.tiff' in datasets:
        sharpened_img_dataset = datasets['sharpened_img_dataset.tiff']
    if 'mask_dataset.tiff' in datasets:
        mask_dataset = datasets['mask_dataset.tiff']
    
    # Now you can proceed with your ImageEditor or other operations.


Loaded image_dataset.tiff from C:/Users/Ding-Yang/Desktop/Thesis code/Axons-extraction/example data/less_intensive\image_dataset.tiff
Loaded sharpened_img_dataset.tiff from C:/Users/Ding-Yang/Desktop/Thesis code/Axons-extraction/example data/less_intensive\sharpened_img_dataset.tiff
Loaded mask_dataset.tiff from C:/Users/Ding-Yang/Desktop/Thesis code/Axons-extraction/example data/less_intensive\mask_dataset.tiff
All required files are loaded.
All files loaded successfully.
